In [1]:
import tweepy
from datetime import datetime
from tqdm.notebook import trange, tqdm
import pickle
import glob

import re
import json
import time
import requests

In [30]:
DATA_PATH = '../data/twitter_reddit/'
TWITTER_DATA_DIR = '../data/twitter_reddit/manual_dataset/twitter/'
REDDIT_DATA_DIR = '../data/twitter_reddit/manual_dataset/reddit/'
REDDIT_SUB_DATA_DIR = '../data/twitter_reddit/manual_dataset/reddit_submissions/'

In [3]:
with open(DATA_PATH + 'to_collect.p', 'rb') as f:
    to_collect, _, _, _, _ = pickle.load(f)

In [4]:
reddit_users = [d[0] for d in to_collect]
twitter_users = [d[1] for d in to_collect]

Download Twitter Data
==

In [5]:

def get_all_tweets(screen_name):
    alltweets = []
    try:
        new_tweets = api.user_timeline(screen_name=screen_name, tweet_mode="extended", count=200)
    except tweepy.RateLimitError:
        print('Rate limitted. Wating...')
        time.sleep(15 * 60)
        
    #save most recent tweets
    alltweets.extend(new_tweets)
    if len(alltweets) == 0:
        return alltweets
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
#         print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        try:
            new_tweets = api.user_timeline(screen_name = screen_name, count=200, max_id=oldest, tweet_mode="extended")
        except tweepy.RateLimitError:
            print('Rate limitted. Wating...')
            time.sleep(15 * 60)
        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

#         print("...%s tweets downloaded so far" % (len(alltweets)))
    return alltweets

In [6]:
auth = tweepy.OAuthHandler('XXXX', 'XXXX')
auth.set_access_token('XXX', 'XXX')
api = tweepy.API(auth)

In [18]:
completed_users = [re.search('/twitter/(.*).jsonl', f).group(1) for f in glob.glob(TWITTER_DATA_DIR + '*.jsonl')]
failed = []

In [ ]:

for u in twitter_users:
    try:
        print(u)
        if u in completed_users or u in failed:
            continue
        all_tweets = get_all_tweets(u)
        tweets = [t for t in all_tweets if not hasattr(t, "retweeted_status")]
        print(u, len(all_tweets), len(tweets))
        completed_users.append(u)
        if len(tweets) < 100:
            continue
        with open(TWITTER_DATA_DIR + u + '.jsonl', 'w') as f:
            for t in tweets:
                json.dump(t.full_text, f)
                f.write('\n')
    except tweepy.RateLimitError:
        time.sleep(15 * 60)
    except Exception as e:
        print('Failed: ', e)
        failed.append(u)

Download Reddit Data
===

In [49]:
def getPushshiftSubmissions(author_name, after, before):
    url = 'https://api.pushshift.io/reddit/search/submission/?author='+str(author_name)+'&size=1000&after='+str(after)+'&before='+str(before)
    r = requests.get(url)
    
    while str(r) == '<Response [429]>':
        time.sleep(10)
        print('waiting')
        r = requests.get(url)
        
    data = json.loads(r.text)
    
    return data['data']


def getPushshiftComMeta(author_name, after, before):
    url = 'https://api.pushshift.io/reddit/search/comment/?author='+str(author_name)+'&size=1000&after='+str(after)+'&before='+str(before)
    r = requests.get(url)
    
    while str(r) == '<Response [429]>':
        time.sleep(10)
        print('waiting')
        r = requests.get(url)
        
    data = json.loads(r.text)
    
    return data['data']

def reddit_user_comments(author_name,after_date,before_date):
    
    data = getPushshiftComMeta(author_name, after_date, before_date)
    
    all_data_user = []

    while len(data) > 0:
        """
            Calls getPushshiftData() with the created date of the last submission
        """
        for com in data:
            all_data_user.append(com['body'])

        ##Change the after
        after = data[-1]['created_utc']
        data = getPushshiftComMeta(author_name, after, before_date)
        print(len(all_data_user))
        if len(all_data_user) > 10000:
            break
        
    return all_data_user


def reddit_user_submissions(author_name,after_date,before_date):
    
    data = getPushshiftSubmissions(author_name, after_date, before_date)
    
    all_data_user = []

    while len(data) > 0:
        """
            Calls getPushshiftData() with the created date of the last submission
        """
        for com in data:
            text = com['title']
            if 'selftext' in com:
                text += '\n' + com['selftext']
            all_data_user.append(text)

        ##Change the after
        after = data[-1]['created_utc']
        data = getPushshiftSubmissions(author_name, after, before_date)
        print(len(all_data_user))
        if len(all_data_user) > 10000:
            break
        
    return all_data_user

In [21]:
after = str(int(datetime(2010, 1, 1).timestamp()))
before = str(int(datetime.now().timestamp()))

In [50]:
completed_users_reddit = [re.search('/reddit/(.*).jsonl', f).group(1) for f in glob.glob(REDDIT_DATA_DIR + '*.jsonl')]
failed_reddit = ['[deleted]']

In [15]:
failed_reddit.append('Trollkitten')

In [ ]:
for u in tqdm(reddit_users):
    try:
        print(u)
        if u in completed_users_reddit or u in failed_reddit or 'Bot' in u:
            continue
        all_comments = reddit_user_comments(u, after, before)
        print(u, len(all_comments))
        completed_users_reddit.append(u)
        if len(all_comments) < 100:
            continue
        with open(REDDIT_DATA_DIR + u + '.jsonl', 'w') as f:
            for c in all_comments:
                json.dump(c, f)
                f.write('\n')
    except Exception as e:
        print('Failed: ', e)
        failed_reddit.append(u)

In [51]:
completed_users_reddit_submissions = [re.search('/reddit_submissions/(.*).jsonl', f).group(1) for f in glob.glob(REDDIT_SUB_DATA_DIR + '*.jsonl')]
failed_reddit_submissions = ['[deleted]']

In [ ]:
for u in tqdm(reddit_users):
    try:
        print(u)
        if u in completed_users_reddit_submissions or u in failed_reddit_submissions or 'bot' in u.lower():
            continue
        all_subs = reddit_user_submissions(u, after, before)
        print(u, len(all_subs))
        completed_users_reddit_submissions.append(u)
        if len(all_subs) < 50:
            continue
        with open(REDDIT_SUB_DATA_DIR + u + '.jsonl', 'w') as f:
            for c in all_subs:
                json.dump(c, f)
                f.write('\n')
    except Exception as e:
        print('Failed: ', e)
        failed_reddit_submissions.append(u)

In [34]:
# [u for u in twitter_users if u in saved_twitter_users]
[u for u in reddit_users if u in saved_reddit_users]

[]

In [35]:

saved_twitter_users = [re.search('/twitter/(.*).jsonl', f).group(1) for f in glob.glob(TWITTER_DATA_DIR + '*.jsonl')]
saved_reddit_users = [re.search('/reddit/(.*).jsonl', f).group(1) for f in glob.glob(REDDIT_DATA_DIR + '*.jsonl')]

reddit_to_twitter = {
    reddit_users[i]:twitter_users[i] 
    for i in range(len(reddit_users)) 
    if reddit_users[i] in saved_reddit_users and twitter_users[i] in saved_twitter_users}

In [36]:
len(reddit_to_twitter)

321